In [1]:
import pandas as pd # 2.x to support pyarrow
import pyarrow as pa
import openpyxl # for reading xlsx with structure
import numpy as np
import plotly.express as px
import math
import random

import nltk
from nltk.corpus import stopwords
from string import punctuation
from lemmagen3 import Lemmatizer
# nltk.download("stopwords")

import pickle
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

In [2]:
lem_ru = Lemmatizer('ru')

ignored_words = list(punctuation) + stopwords.words("russian") + ['№', '``', '\'\'']
def extract_words(text):
    tokens = nltk.word_tokenize(text.replace('/', ' ').replace('\\', ' ').replace('.', ' ').replace('_', ' ').replace('=', ' '))
    words = [token.lower().strip() for token in tokens]
    clean_words = [lem_ru.lemmatize(word) for word in words if word not in ignored_words]
    return clean_words

In [3]:
train_data = pd.read_csv('../data/train.csv', parse_dates=['Крайний срок', 'Дата обращения', 'Дата восстановления', 'Дата закрытия обращения'])
train_data['Решение'] = train_data['Решение             ']
del train_data['Решение             ']

In [4]:
train_data

,Содержание,Сервис,Приоритет,Статус,Функциональная группа,Крайний срок,Дата обращения,Тип обращения на момент подачи,Тип обращения итоговый,Тип переклассификации,Дата восстановления,Дата закрытия обращения,Критичность,Влияние,Система,Место,Решение
0,Тест,АРМ,3-Низкий,Отменен,ФГ1,2018-01-24 07:42:00,2018-01-19 09:27:00,Запрос,Запрос,0,NaT,2018-01-19 09:43:00,4-Нет,3-Малое,База,К,Тест
1,Тестовое обращение,АРМ,3-Низкий,Отменен,ФГ1,2018-01-24 10:51:00,2018-01-19 09:36:00,Запрос,Запрос,0,NaT,2018-01-19 09:41:00,4-Нет,3-Малое,База,К,Тест
2,Выполнена разблокировка у/з REGION\*****,Сервис7,2-Средний,Закрыт,ФГ8,2018-01-31 07:51:00,2018-01-25 22:47:00,Запрос,Запрос,0,2018-01-25 22:47:00,2018-02-02 07:52:00,1-Особая,3-Малое,Система7,Место190,Выполнена разблокировка у/з Ne_bosS\*********\...
3,.Не включается ПК,АРМ,3-Низкий,Закрыт,ФГ390,2018-01-28 05:50:00,2018-01-25 23:13:00,Инцидент,Инцидент,0,2018-01-25 23:55:00,2018-01-26 00:00:00,4-Нет,3-Малое,База,Место177,Выход из стоя БП. Заменён.
4,"Заявка № ***********, *******************",АРМ,3-Низкий,Закрыт,ФГ303,2018-02-03 02:51:00,2018-01-25 23:39:00,Запрос,Запрос,0,2018-01-28 02:54:00,2018-02-04 07:53:00,4-Нет,4-Нет влияния,База,Место216,Работы по обращению выполнены
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14798,разблокировка учетной записи пользователя вСе...,Сервис7,2-Средний,Закрыт,ФГ8,2018-02-21 14:17:00,2018-02-16 13:02:00,Запрос,Запрос,0,2018-02-16 13:02:00,2018-02-23 14:18:00,1-Особая,3-Малое,Система7,Место50,Личность пользователя идентифицирована\nПричи...
14799,Устранение уязвимостей ИБ в соответствии с Пла...,АРМ,3-Низкий,Закрыт,ФГ135,2018-02-21 14:36:00,2018-02-16 13:07:00,Запрос,Запрос,0,2018-02-21 12:03:00,2018-03-07 13:22:00,4-Нет,3-Малое,База,Место3,Выполнено
14800,Контроль ПУ. Заявка на доступ к портам.,Сервис2,3-Низкий,Закрыт,ФГ315,2018-02-21 14:23:00,2018-02-16 13:08:00,Запрос,Запрос,0,2018-02-21 07:01:00,2018-03-07 08:46:00,4-Нет,4-Нет влияния,Система2,Место163,Работы выполнены согласно заявке.
14801,Ремонт стульев.,Сервис73,3-Низкий,Закрыт,ФГ159,2018-02-21 14:24:00,2018-02-16 13:09:00,Запрос,Запрос,0,2018-02-17 10:01:00,2018-02-24 11:22:00,4-Нет,4-Нет влияния,Система71,Место3,выполнено


In [5]:
train_data_clean = train_data.copy().dropna()

train_data_clean['Содержание'] = train_data_clean['Содержание'].map(lambda x: ' '.join(extract_words(x)))
train_data_clean['Решение'] = train_data_clean['Решение'].map(lambda x: ' '.join(extract_words(x)))
# train_data_clean.to_csv('../data/train_clean.csv', index=False)
train_data_clean

,Содержание,Сервис,Приоритет,Статус,Функциональная группа,Крайний срок,Дата обращения,Тип обращения на момент подачи,Тип обращения итоговый,Тип переклассификации,Дата восстановления,Дата закрытия обращения,Критичность,Влияние,Система,Место,Решение
2,выполнять разблокировка без region,Сервис7,2-Средний,Закрыт,ФГ8,2018-01-31 07:51:00,2018-01-25 22:47:00,Запрос,Запрос,0,2018-01-25 22:47:00,2018-02-02 07:52:00,1-Особая,3-Малое,Система7,Место190,выполнять разблокировка без ne boss идентифика...
3,включаться пк,АРМ,3-Низкий,Закрыт,ФГ390,2018-01-28 05:50:00,2018-01-25 23:13:00,Инцидент,Инцидент,0,2018-01-25 23:55:00,2018-01-26 00:00:00,4-Нет,3-Малое,База,Место177,выход стоить бп заменять
4,заявка,АРМ,3-Низкий,Закрыт,ФГ303,2018-02-03 02:51:00,2018-01-25 23:39:00,Запрос,Запрос,0,2018-01-28 02:54:00,2018-02-04 07:53:00,4-Нет,4-Нет влияния,База,Место216,работа обращение выполнять
5,заявка,АРМ,3-Низкий,Закрыт,ФГ372,2018-01-31 07:51:00,2018-01-25 23:39:00,Запрос,Запрос,0,2018-01-26 03:19:00,2018-02-02 07:52:00,4-Нет,4-Нет влияния,Система62,Место216,доступ заблокировать
6,заявка,АРМ,3-Низкий,Закрыт,ФГ303,2018-02-01 02:51:00,2018-01-25 23:39:00,Запрос,Запрос,0,2018-01-26 03:11:00,2018-02-02 07:52:00,4-Нет,4-Нет влияния,База,Место216,работа обращение выполнять
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14798,разблокировка учетный запись пользователь всер...,Сервис7,2-Средний,Закрыт,ФГ8,2018-02-21 14:17:00,2018-02-16 13:02:00,Запрос,Запрос,0,2018-02-16 13:02:00,2018-02-23 14:18:00,1-Особая,3-Малое,Система7,Место50,личность пользователь идентифицировать причина...
14799,устранение уязвимость иб соответствие планом№,АРМ,3-Низкий,Закрыт,ФГ135,2018-02-21 14:36:00,2018-02-16 13:07:00,Запрос,Запрос,0,2018-02-21 12:03:00,2018-03-07 13:22:00,4-Нет,3-Малое,База,Место3,выполнять
14800,контроль пу заявка доступ порт,Сервис2,3-Низкий,Закрыт,ФГ315,2018-02-21 14:23:00,2018-02-16 13:08:00,Запрос,Запрос,0,2018-02-21 07:01:00,2018-03-07 08:46:00,4-Нет,4-Нет влияния,Система2,Место163,работа выполнять согласно заявка
14801,ремонт стул,Сервис73,3-Низкий,Закрыт,ФГ159,2018-02-21 14:24:00,2018-02-16 13:09:00,Запрос,Запрос,0,2018-02-17 10:01:00,2018-02-24 11:22:00,4-Нет,4-Нет влияния,Система71,Место3,выполнять


In [6]:
test_data = pd.read_csv('../data/test.csv', parse_dates=['Крайний срок', 'Дата обращения', 'Дата восстановления', 'Дата закрытия обращения'])
test_data['Решение'] = test_data['Решение             ']
del test_data['Решение             ']
del test_data['Тип обращения итоговый']
del test_data['Тип переклассификации']

In [7]:
test_data

,id,Содержание,Сервис,Приоритет,Статус,Функциональная группа,Крайний срок,Дата обращения,Тип обращения на момент подачи,Дата восстановления,Дата закрытия обращения,Критичность,Влияние,Система,Место,Решение
0,14803,Заявка на предоставление и отзыв прав доступа ...,Сервис41,3-Низкий,Закрыт,ФГ13,2018-02-22 07:51:00,2018-02-16 13:11:00,Запрос,2018-02-18 04:32:00,2018-02-25 07:56:00,4-Нет,4-Нет влияния,Система43,Место38,Работы по обращению выполнены
1,14804,Транспортный шлюз,Сервис61,2-Средний,Закрыт,ФГ1,2018-02-21 14:36:00,2018-02-16 13:13:00,Запрос,2018-02-16 17:55:00,2018-02-24 07:54:00,1-Особая,3-Малое,Система61,К,проинф у.о.
2,14805,Заявка на предоставление и отзыв прав доступа ...,Сервис22,3-Низкий,Закрыт,ФГ13,2018-03-31 06:36:00,2018-02-16 13:13:00,Запрос,2018-03-15 16:09:00,2018-03-23 07:56:00,2-Повышенная,4-Нет влияния,Система13,Место25,Работы по обращению выполнены
3,14806,Заявка на предоставление и отзыв прав доступа ...,Сервис91,3-Низкий,Отменен,ФГ13,2018-02-21 14:29:00,2018-02-16 13:14:00,Запрос,2018-02-16 13:15:00,NaT,2-Повышенная,4-Нет влияния,Система363,Место1,Запрос отменён пользователем из портала самооб...
4,14807,Заявка на предоставление и отзыв прав доступа ...,Сервис22,3-Низкий,Закрыт,ФГ13,2018-03-30 09:36:00,2018-02-16 13:14:00,Запрос,2018-03-15 16:09:00,2018-03-23 07:56:00,2-Повышенная,4-Нет влияния,Система13,Место25,Работы по обращению выполнены
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339,21142,<Бот-наблюдения> Система53. Ухудшение статуса ...,Сервис53,3-Низкий,Закрыт,ФГ4,2018-03-07 14:36:00,2018-02-25 20:56:00,Инцидент,2018-02-26 05:57:00,2018-03-14 07:53:00,2-Повышенная,3-Малое,Система53,Место1,самовосстановление
6340,21143,<Бот-наблюдения> Система52. Ухудшение статуса ...,Сервис52,2-Средний,Закрыт,ФГ4,2018-03-07 14:06:00,2018-02-25 20:58:00,Инцидент,2018-02-26 06:00:00,2018-03-14 07:53:00,1-Особая,3-Малое,Система52,Место1,самовосстановление
6341,21144,<Бот-наблюдения> Система11. Ухудшение статуса ...,Сервис11,3-Низкий,Закрыт,ФГ4,2018-03-09 12:36:00,2018-02-25 21:05:00,Инцидент,2018-02-26 06:03:00,2018-03-14 07:53:00,4-Нет,3-Малое,Система11,Место1,самовосстановление
6342,21145,<Бот-наблюдения>Система364. Ухудшение статуса ...,Сервис4,3-Низкий,Закрыт,ФГ5,2018-03-09 12:36:00,2018-02-25 21:16:00,Инцидент,2018-03-07 09:28:00,2018-03-14 10:43:00,2-Повышенная,3-Малое,Система4,Место1,Самовосстановление


In [8]:
test_data_clean = test_data.copy()
test_data_clean['Содержание'] = test_data_clean['Содержание'].fillna('').map(lambda x: ' '.join(extract_words(x)))
test_data_clean['Решение'] = test_data_clean['Решение'].fillna('').map(lambda x: ' '.join(extract_words(x)))
test_data_clean

,id,Содержание,Сервис,Приоритет,Статус,Функциональная группа,Крайний срок,Дата обращения,Тип обращения на момент подачи,Дата восстановления,Дата закрытия обращения,Критичность,Влияние,Система,Место,Решение
0,14803,заявка предоставление отзыв правый доступ ресурс,Сервис41,3-Низкий,Закрыт,ФГ13,2018-02-22 07:51:00,2018-02-16 13:11:00,Запрос,2018-02-18 04:32:00,2018-02-25 07:56:00,4-Нет,4-Нет влияния,Система43,Место38,работа обращение выполнять
1,14804,транспортный шлюз,Сервис61,2-Средний,Закрыт,ФГ1,2018-02-21 14:36:00,2018-02-16 13:13:00,Запрос,2018-02-16 17:55:00,2018-02-24 07:54:00,1-Особая,3-Малое,Система61,К,проинф
2,14805,заявка предоставление отзыв правый доступ ресурс,Сервис22,3-Низкий,Закрыт,ФГ13,2018-03-31 06:36:00,2018-02-16 13:13:00,Запрос,2018-03-15 16:09:00,2018-03-23 07:56:00,2-Повышенная,4-Нет влияния,Система13,Место25,работа обращение выполнять
3,14806,заявка предоставление отзыв правый доступ ресурс,Сервис91,3-Низкий,Отменен,ФГ13,2018-02-21 14:29:00,2018-02-16 13:14:00,Запрос,2018-02-16 13:15:00,NaT,2-Повышенная,4-Нет влияния,Система363,Место1,запрос отменять пользователь портал самообслуж...
4,14807,заявка предоставление отзыв правый доступ ресурс,Сервис22,3-Низкий,Закрыт,ФГ13,2018-03-30 09:36:00,2018-02-16 13:14:00,Запрос,2018-03-15 16:09:00,2018-03-23 07:56:00,2-Повышенная,4-Нет влияния,Система13,Место25,работа обращение выполнять
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339,21142,бот-наблюдение система53 ухудшение статус сервис,Сервис53,3-Низкий,Закрыт,ФГ4,2018-03-07 14:36:00,2018-02-25 20:56:00,Инцидент,2018-02-26 05:57:00,2018-03-14 07:53:00,2-Повышенная,3-Малое,Система53,Место1,самовосстановление
6340,21143,бот-наблюдение система52 ухудшение статус сервис,Сервис52,2-Средний,Закрыт,ФГ4,2018-03-07 14:06:00,2018-02-25 20:58:00,Инцидент,2018-02-26 06:00:00,2018-03-14 07:53:00,1-Особая,3-Малое,Система52,Место1,самовосстановление
6341,21144,бот-наблюдение система11 ухудшение статус сервис,Сервис11,3-Низкий,Закрыт,ФГ4,2018-03-09 12:36:00,2018-02-25 21:05:00,Инцидент,2018-02-26 06:03:00,2018-03-14 07:53:00,4-Нет,3-Малое,Система11,Место1,самовосстановление
6342,21145,бот-наблюдение система364 ухудшение статус сервис,Сервис4,3-Низкий,Закрыт,ФГ5,2018-03-09 12:36:00,2018-02-25 21:16:00,Инцидент,2018-03-07 09:28:00,2018-03-14 10:43:00,2-Повышенная,3-Малое,Система4,Место1,самовосстановление


In [9]:
# @NOTE: Словари зафиксированы, чтобы были воспроизводимые значения index embedding
#        В реальных условиях нужно включить в словари все возможные реальные значения и переобучить нейронную сеть

service_dictionary = ['АРМ', 'Сервис1', 'Сервис10', 'Сервис100', 'Сервис102', 'Сервис104', 'Сервис105', 'Сервис106', 'Сервис107', 'Сервис108', 'Сервис11', 'Сервис113', 'Сервис115', 'Сервис116', 'Сервис117', 'Сервис118', 'Сервис119', 'Сервис12', 'Сервис120', 'Сервис121', 'Сервис122', 'Сервис124', 'Сервис125', 'Сервис126', 'Сервис127', 'Сервис129', 'Сервис13', 'Сервис130', 'Сервис131', 'Сервис132', 'Сервис133', 'Сервис135', 'Сервис136', 'Сервис137', 'Сервис138', 'Сервис140', 'Сервис142', 'Сервис149', 'Сервис151', 'Сервис152', 'Сервис153', 'Сервис154', 'Сервис155', 'Сервис156', 'Сервис158', 'Сервис159', 'Сервис16', 'Сервис161', 'Сервис17', 'Сервис171', 'Сервис174', 'Сервис177', 'Сервис178', 'Сервис18', 'Сервис180', 'Сервис181', 'Сервис183', 'Сервис185', 'Сервис19', 'Сервис191', 'Сервис192', 'Сервис193', 'Сервис194', 'Сервис198', 'Сервис2', 'Сервис20', 'Сервис201', 'Сервис205', 'Сервис206', 'Сервис207', 'Сервис208', 'Сервис21', 'Сервис210', 'Сервис213', 'Сервис216', 'Сервис22', 'Сервис220', 'Сервис221', 'Сервис223', 'Сервис224', 'Сервис23', 'Сервис231', 'Сервис235', 'Сервис240', 'Сервис245', 'Сервис247', 'Сервис25', 'Сервис256', 'Сервис257', 'Сервис258', 'Сервис259', 'Сервис26', 'Сервис269', 'Сервис27', 'Сервис276', 'Сервис28', 'Сервис286', 'Сервис287', 'Сервис29', 'Сервис293', 'Сервис294', 'Сервис295', 'Сервис3', 'Сервис30', 'Сервис31', 'Сервис310', 'Сервис311', 'Сервис314', 'Сервис316', 'Сервис320', 'Сервис33', 'Сервис336', 'Сервис337', 'Сервис34', 'Сервис340', 'Сервис347', 'Сервис36', 'Сервис366', 'Сервис37', 'Сервис375', 'Сервис38', 'Сервис39', 'Сервис390', 'Сервис4', 'Сервис40', 'Сервис401', 'Сервис41', 'Сервис415', 'Сервис42', 'Сервис43', 'Сервис44', 'Сервис45', 'Сервис46', 'Сервис47', 'Сервис48', 'Сервис5', 'Сервис50', 'Сервис51', 'Сервис52', 'Сервис53', 'Сервис54', 'Сервис55', 'Сервис56', 'Сервис57', 'Сервис58', 'Сервис59', 'Сервис6', 'Сервис60', 'Сервис61', 'Сервис62', 'Сервис64', 'Сервис65', 'Сервис66', 'Сервис7', 'Сервис70', 'Сервис71', 'Сервис72', 'Сервис73', 'Сервис75', 'Сервис76', 'Сервис77', 'Сервис78', 'Сервис8', 'Сервис81', 'Сервис84', 'Сервис86', 'Сервис87', 'Сервис88', 'Сервис9', 'Сервис90', 'Сервис91', 'Сервис92', 'Сервис93', 'Сервис94', 'Сервис95', 'Сервис98', 'Сервис99']

status_dictionary = ['Отменен', 'Закрыт']

fg_dictionary = ['СВ12', 'СВ22', 'СВ28', 'СВ3', 'СВ31', 'СВ4', 'СВ6', 'ФГ1', 'ФГ10', 'ФГ100', 'ФГ1008', 'ФГ101', 'ФГ102', 'ФГ103', 'ФГ104', 'ФГ105', 'ФГ106', 'ФГ108', 'ФГ109', 'ФГ11', 'ФГ110', 'ФГ112', 'ФГ113', 'ФГ114', 'ФГ115', 'ФГ116', 'ФГ117', 'ФГ119', 'ФГ12', 'ФГ120', 'ФГ121', 'ФГ122', 'ФГ124', 'ФГ125', 'ФГ126', 'ФГ128', 'ФГ129', 'ФГ13', 'ФГ130', 'ФГ131', 'ФГ133', 'ФГ134', 'ФГ135', 'ФГ136', 'ФГ137', 'ФГ139', 'ФГ14', 'ФГ140', 'ФГ141', 'ФГ142', 'ФГ143', 'ФГ146', 'ФГ147', 'ФГ148', 'ФГ149', 'ФГ151', 'ФГ152', 'ФГ153', 'ФГ154', 'ФГ155', 'ФГ156', 'ФГ157', 'ФГ159', 'ФГ160', 'ФГ161', 'ФГ162', 'ФГ163', 'ФГ164', 'ФГ165', 'ФГ166', 'ФГ167', 'ФГ169', 'ФГ17', 'ФГ170', 'ФГ171', 'ФГ172', 'ФГ173', 'ФГ174', 'ФГ175', 'ФГ176', 'ФГ177', 'ФГ179', 'ФГ18', 'ФГ181', 'ФГ182', 'ФГ183', 'ФГ184', 'ФГ185', 'ФГ186', 'ФГ188', 'ФГ189', 'ФГ19', 'ФГ190', 'ФГ191', 'ФГ192', 'ФГ193', 'ФГ196', 'ФГ198', 'ФГ2', 'ФГ20', 'ФГ201', 'ФГ202', 'ФГ204', 'ФГ205', 'ФГ206', 'ФГ207', 'ФГ208', 'ФГ209', 'ФГ21', 'ФГ211', 'ФГ212', 'ФГ213', 'ФГ214', 'ФГ215', 'ФГ216', 'ФГ217', 'ФГ218', 'ФГ219', 'ФГ22', 'ФГ220', 'ФГ221', 'ФГ222', 'ФГ223', 'ФГ224', 'ФГ226', 'ФГ227', 'ФГ228', 'ФГ229', 'ФГ23', 'ФГ230', 'ФГ232', 'ФГ233', 'ФГ234', 'ФГ235', 'ФГ236', 'ФГ237', 'ФГ238', 'ФГ239', 'ФГ24', 'ФГ240', 'ФГ241', 'ФГ243', 'ФГ244', 'ФГ245', 'ФГ246', 'ФГ247', 'ФГ248', 'ФГ249', 'ФГ25', 'ФГ250', 'ФГ252', 'ФГ253', 'ФГ254', 'ФГ255', 'ФГ256', 'ФГ259', 'ФГ26', 'ФГ262', 'ФГ263', 'ФГ264', 'ФГ266', 'ФГ270', 'ФГ271', 'ФГ272', 'ФГ274', 'ФГ277', 'ФГ279', 'ФГ28', 'ФГ280', 'ФГ281', 'ФГ282', 'ФГ283', 'ФГ284', 'ФГ286', 'ФГ287', 'ФГ289', 'ФГ29', 'ФГ290', 'ФГ291', 'ФГ292', 'ФГ293', 'ФГ294', 'ФГ295', 'ФГ296', 'ФГ297', 'ФГ298', 'ФГ3', 'ФГ30', 'ФГ300', 'ФГ301', 'ФГ302', 'ФГ303', 'ФГ305', 'ФГ306', 'ФГ307', 'ФГ308', 'ФГ309', 'ФГ31', 'ФГ310', 'ФГ312', 'ФГ313', 'ФГ315', 'ФГ316', 'ФГ317', 'ФГ318', 'ФГ319', 'ФГ32', 'ФГ320', 'ФГ321', 'ФГ323', 'ФГ324', 'ФГ325', 'ФГ328', 'ФГ329', 'ФГ33', 'ФГ330', 'ФГ332', 'ФГ333', 'ФГ334', 'ФГ335', 'ФГ336', 'ФГ339', 'ФГ34', 'ФГ340', 'ФГ341', 'ФГ342', 'ФГ343', 'ФГ344', 'ФГ345', 'ФГ346', 'ФГ347', 'ФГ348', 'ФГ35', 'ФГ350', 'ФГ352', 'ФГ353', 'ФГ354', 'ФГ355', 'ФГ356', 'ФГ357', 'ФГ358', 'ФГ359', 'ФГ36', 'ФГ360', 'ФГ361', 'ФГ362', 'ФГ363', 'ФГ366', 'ФГ367', 'ФГ368', 'ФГ369', 'ФГ37', 'ФГ370', 'ФГ371', 'ФГ372', 'ФГ373', 'ФГ374', 'ФГ375', 'ФГ376', 'ФГ377', 'ФГ378', 'ФГ38', 'ФГ381', 'ФГ382', 'ФГ383', 'ФГ384', 'ФГ385', 'ФГ386', 'ФГ387', 'ФГ388', 'ФГ389', 'ФГ39', 'ФГ390', 'ФГ391', 'ФГ392', 'ФГ395', 'ФГ396', 'ФГ397', 'ФГ398', 'ФГ399', 'ФГ4', 'ФГ40', 'ФГ400', 'ФГ401', 'ФГ403', 'ФГ407', 'ФГ408', 'ФГ409', 'ФГ41', 'ФГ413', 'ФГ414', 'ФГ415', 'ФГ418', 'ФГ419', 'ФГ42', 'ФГ421', 'ФГ424', 'ФГ426', 'ФГ428', 'ФГ43', 'ФГ430', 'ФГ431', 'ФГ433', 'ФГ434', 'ФГ437', 'ФГ438', 'ФГ439', 'ФГ44', 'ФГ442', 'ФГ443', 'ФГ444', 'ФГ445', 'ФГ447', 'ФГ448', 'ФГ449', 'ФГ45', 'ФГ456', 'ФГ457', 'ФГ459', 'ФГ46', 'ФГ460', 'ФГ463', 'ФГ464', 'ФГ465', 'ФГ467', 'ФГ468', 'ФГ469', 'ФГ47', 'ФГ470', 'ФГ471', 'ФГ472', 'ФГ478', 'ФГ479', 'ФГ48', 'ФГ480', 'ФГ481', 'ФГ484', 'ФГ485', 'ФГ486', 'ФГ487', 'ФГ489', 'ФГ49', 'ФГ492', 'ФГ493', 'ФГ494', 'ФГ495', 'ФГ496', 'ФГ498', 'ФГ499', 'ФГ5', 'ФГ50', 'ФГ501', 'ФГ502', 'ФГ505', 'ФГ506', 'ФГ507', 'ФГ509', 'ФГ51', 'ФГ510', 'ФГ511', 'ФГ514', 'ФГ515', 'ФГ516', 'ФГ517', 'ФГ519', 'ФГ52', 'ФГ520', 'ФГ521', 'ФГ522', 'ФГ523', 'ФГ524', 'ФГ526', 'ФГ527', 'ФГ528', 'ФГ53', 'ФГ530', 'ФГ531', 'ФГ535', 'ФГ536', 'ФГ538', 'ФГ539', 'ФГ540', 'ФГ546', 'ФГ547', 'ФГ548', 'ФГ549', 'ФГ55', 'ФГ550', 'ФГ552', 'ФГ554', 'ФГ555', 'ФГ558', 'ФГ559', 'ФГ56', 'ФГ562', 'ФГ563', 'ФГ564', 'ФГ565', 'ФГ566', 'ФГ567', 'ФГ569', 'ФГ57', 'ФГ570', 'ФГ571', 'ФГ572', 'ФГ573', 'ФГ574', 'ФГ575', 'ФГ576', 'ФГ577', 'ФГ579', 'ФГ58', 'ФГ580', 'ФГ581', 'ФГ582', 'ФГ584', 'ФГ587', 'ФГ588', 'ФГ589', 'ФГ59', 'ФГ590', 'ФГ592', 'ФГ593', 'ФГ594', 'ФГ597', 'ФГ598', 'ФГ599', 'ФГ6', 'ФГ600', 'ФГ601', 'ФГ603', 'ФГ604', 'ФГ605', 'ФГ608', 'ФГ61', 'ФГ610', 'ФГ611', 'ФГ612', 'ФГ614', 'ФГ615', 'ФГ616', 'ФГ617', 'ФГ619', 'ФГ62', 'ФГ620', 'ФГ624', 'ФГ626', 'ФГ628', 'ФГ629', 'ФГ63', 'ФГ630', 'ФГ633', 'ФГ634', 'ФГ638', 'ФГ639', 'ФГ64', 'ФГ641', 'ФГ649', 'ФГ65', 'ФГ650', 'ФГ651', 'ФГ653', 'ФГ655', 'ФГ66', 'ФГ661', 'ФГ662', 'ФГ665', 'ФГ667', 'ФГ668', 'ФГ669', 'ФГ670', 'ФГ671', 'ФГ672', 'ФГ676', 'ФГ679', 'ФГ68', 'ФГ684', 'ФГ685', 'ФГ686', 'ФГ69', 'ФГ694', 'ФГ695', 'ФГ7', 'ФГ70', 'ФГ700', 'ФГ702', 'ФГ703', 'ФГ705', 'ФГ71', 'ФГ710', 'ФГ716', 'ФГ72', 'ФГ720', 'ФГ721', 'ФГ726', 'ФГ727', 'ФГ728', 'ФГ73', 'ФГ74', 'ФГ740', 'ФГ741', 'ФГ742', 'ФГ743', 'ФГ745', 'ФГ747', 'ФГ748', 'ФГ75', 'ФГ751', 'ФГ754', 'ФГ756', 'ФГ76', 'ФГ761', 'ФГ762', 'ФГ764', 'ФГ766', 'ФГ77', 'ФГ770', 'ФГ771', 'ФГ772', 'ФГ774', 'ФГ78', 'ФГ780', 'ФГ781', 'ФГ784', 'ФГ785', 'ФГ786', 'ФГ787', 'ФГ789', 'ФГ79', 'ФГ791', 'ФГ799', 'ФГ8', 'ФГ80', 'ФГ800', 'ФГ804', 'ФГ809', 'ФГ81', 'ФГ811', 'ФГ816', 'ФГ82', 'ФГ822', 'ФГ823', 'ФГ824', 'ФГ825', 'ФГ827', 'ФГ83', 'ФГ832', 'ФГ834', 'ФГ84', 'ФГ841', 'ФГ845', 'ФГ847', 'ФГ85', 'ФГ856', 'ФГ858', 'ФГ86', 'ФГ864', 'ФГ869', 'ФГ87', 'ФГ871', 'ФГ876', 'ФГ883', 'ФГ887', 'ФГ889', 'ФГ89', 'ФГ890', 'ФГ898', 'ФГ9', 'ФГ903', 'ФГ906', 'ФГ91', 'ФГ910', 'ФГ911', 'ФГ919', 'ФГ926', 'ФГ928', 'ФГ94', 'ФГ947', 'ФГ949', 'ФГ95', 'ФГ953', 'ФГ955', 'ФГ96', 'ФГ961', 'ФГ963', 'ФГ967', 'ФГ97', 'ФГ98', 'ФГ983', 'ФГ988', 'ФГ99', 'ФГ991', 'ФГ993', 'ФГСистема268']

type_dictionary = ['Запрос', 'Инцидент']

system_dictionary = ['База', 'Система1', 'Система10', 'Система100', 'Система101', 'Система102', 'Система104', 'Система105', 'Система107', 'Система108', 'Система109', 'Система11', 'Система110', 'Система111', 'Система112', 'Система113', 'Система115', 'Система116', 'Система117', 'Система118', 'Система119', 'Система12', 'Система120', 'Система121', 'Система122', 'Система123', 'Система124', 'Система125', 'Система126', 'Система129', 'Система13', 'Система130', 'Система131', 'Система133', 'Система134', 'Система135', 'Система136', 'Система137', 'Система138', 'Система139', 'Система140', 'Система141', 'Система143', 'Система144', 'Система145', 'Система146', 'Система147', 'Система148', 'Система149', 'Система151', 'Система153', 'Система156', 'Система158', 'Система159', 'Система16', 'Система160', 'Система161', 'Система162', 'Система164', 'Система165', 'Система166', 'Система168', 'Система169', 'Система17', 'Система170', 'Система173', 'Система175', 'Система18', 'Система180', 'Система184', 'Система188', 'Система189', 'Система19', 'Система190', 'Система193', 'Система194', 'Система195', 'Система198', 'Система2', 'Система20', 'Система200', 'Система205', 'Система206', 'Система207', 'Система208', 'Система21', 'Система210', 'Система212', 'Система214', 'Система218', 'Система219', 'Система22', 'Система221', 'Система224', 'Система225', 'Система226', 'Система228', 'Система23', 'Система234', 'Система24', 'Система241', 'Система243', 'Система244', 'Система245', 'Система246', 'Система247', 'Система25', 'Система257', 'Система26', 'Система262', 'Система266', 'Система27', 'Система275', 'Система278', 'Система28', 'Система287', 'Система288', 'Система289', 'Система29', 'Система290', 'Система299', 'Система3', 'Система30', 'Система303', 'Система304', 'Система306', 'Система309', 'Система31', 'Система312', 'Система318', 'Система319', 'Система321', 'Система328', 'Система329', 'Система33', 'Система332', 'Система333', 'Система334', 'Система336', 'Система34', 'Система341', 'Система348', 'Система35', 'Система353', 'Система356', 'Система36', 'Система363', 'Система368', 'Система373', 'Система375', 'Система38', 'Система386', 'Система389', 'Система39', 'Система391', 'Система394', 'Система395', 'Система4', 'Система40', 'Система404', 'Система41', 'Система410', 'Система419', 'Система42', 'Система422', 'Система43', 'Система438', 'Система44', 'Система441', 'Система45', 'Система453', 'Система46', 'Система462', 'Система466', 'Система469', 'Система47', 'Система470', 'Система473', 'Система477', 'Система478', 'Система48', 'Система482', 'Система483', 'Система489', 'Система5', 'Система50', 'Система51', 'Система52', 'Система53', 'Система54', 'Система55', 'Система56', 'Система57', 'Система58', 'Система6', 'Система60', 'Система61', 'Система62', 'Система64', 'Система65', 'Система68', 'Система69', 'Система7', 'Система70', 'Система71', 'Система73', 'Система75', 'Система76', 'Система77', 'Система8', 'Система80', 'Система81', 'Система84', 'Система86', 'Система87', 'Система88', 'Система9', 'Система90', 'Система91', 'Система92', 'Система93', 'Система94', 'Система95', 'Система97', 'Система99', 'СистемаСистема268']

location_dictionary = ['#Н/Д', 'Место190', 'Место177', 'Место216', 'Место19', 'Место257', 'Место1', 'Место222', 'Место172', 'Место159', 'Место143', 'Место181', 'Место106', 'Место191', 'Место173', 'Место141', 'Место146', 'Место149', 'Место67', 'Место18', 'Место164', 'Место157', 'Место161', 'Место155', 'Место274', 'Место20', 'К', 'Место56', 'Место41', 'Место369', 'Н', 'Место160', 'Место114', 'Место15', 'Место201', 'Место163', 'Место200', 'Место13', 'Место174', 'Место122', 'Место58', 'Место89', 'Место77', 'Место63', 'Место92', 'Место321', 'Место76', 'Место206', 'Место42', 'Место283', 'Место57', 'Место103', 'Место197', 'Место66', 'Место7', 'Место85', 'Место25', 'Место86', 'Место75', 'Место388', 'Место32', 'Место38', 'Место16', 'Место59', 'Место51', 'Место24', 'Место170', 'Место116', 'Место83', 'Место125', 'Место6', 'Место27', 'Место117', 'Место3', 'Место210', 'Место50', 'Место150', 'Место2', 'Место179', 'Место140', 'Место416', 'Место97', 'Место65', 'Место5', 'Н/Д', 'Место198', 'Место33', 'Место31', 'Место156', 'Место168', 'Место236', 'Место30', 'Место82', 'Место128', 'Место37', 'Место26', 'Место119', 'Место93', 'Место44', 'Место353', 'Место55', 'Место28', 'Место240', 'Место102', 'Место84', 'Место14', 'Место73', 'Место11', 'Место165', 'Место10', 'Место101', 'Место39', 'Место48', 'Место91', 'Место9', 'Место69', 'Место8', 'Место145', 'Место64', 'Место17', 'Место45', 'Место137', 'Место139', 'Место79', 'Место415', 'Место207', 'Место330', 'Место333', 'Место340', 'Место21', 'Место34', 'Место115', 'Место53', 'Место47', 'Место224', 'Место169', 'Место78', 'Место134', 'Место22', 'Место95', 'Место253', 'Место148', 'Место306', 'Место49', 'Место123', 'Место404', 'Место373', 'Место110', 'Место52', 'Место68', 'Место90', 'Место4', 'Место54', 'Место118', 'Место105', 'Место109', 'Место167', 'Место23', 'Место220', 'Место287', 'Место218', 'Место217', 'Место185', 'Место203', 'Место194', 'Место202', 'Место235', 'Место113', 'Место208', 'Место129', 'Место43', 'Место213', 'Место407', 'Место394', 'Место260', 'Место225', 'Место307', 'Место317', 'Место175', 'Место152', 'Место100', 'Место61', 'Место104', 'Место183', 'Место234', 'Место126', 'Место178', 'Место199', 'Место35', 'Место285', 'Место136', 'Место46', 'Место238', 'Место74', 'Место132', 'Место261', 'Место254', 'Место341', 'Место29', 'Место72', 'Место281', 'Место223', 'Место266', 'Место279', 'Место215', 'Место127', 'Место350', 'Место302', 'Место227', 'Место120', 'Место36', 'Место99', 'Место299', 'Место62', 'Место212', 'Место277', 'Место144', 'Место308', 'Место226', 'Место147', 'Место292', 'Место219', 'Место243', 'Место271', 'Место326', 'Место258', 'Место70', 'Место182', 'Место366', 'Место269', 'Место264', 'Место231', 'Место359', 'Место297', 'Место315', 'Место133', 'Место375', 'Место325', 'Место294', 'Место244', 'Место406', 'Место405', 'Место80', 'Место248', 'Место291', 'Место176', 'Место304', 'Место336', 'Место239', 'Место382', 'Место241', 'Место255', 'Место98', 'Место237', 'Место249', 'Место370', 'Место298', 'Место232', 'Место371', 'Место108', 'Место40', 'Место368', 'Место221', 'Место301', 'Место154', 'Место96', 'Место335', 'Место94', 'Место247', 'Место60', 'Место71', 'Место153', 'Место289', 'Место87', 'Место316', 'Место337', 'Место319', 'Место205', 'Место209', 'Место343', 'Место329', 'Место158', 'Место270', 'Место259', 'Место393', 'Место162', 'Место331', 'Место303', 'Место138', 'Место184', 'Место192', 'Место352', 'Место88', 'Место323', 'Место387', 'Место112', 'Место284', 'Место320', 'Место228', 'Место229', 'Место272', 'Место180', 'Место12', 'Место142', 'Место318', 'Место193', 'Место263', 'Место381', 'Место354', 'Место130', 'Место214', 'Место251', 'Место282', 'Место186', 'Место334', 'Место363', 'Место295', 'Место362', 'Место342', 'Место356', 'МестоСистема268', 'Место347', 'Место349', 'Место345', 'Место135', 'Место348', 'Место339', 'Место81', 'Место314', 'Место250', 'Место311', 'Место305', 'Место204', 'Место300', 'Место293', 'Место171', 'Место256', 'Место131', 'Место246', 'Место211', 'Место187', 'Место230']

In [10]:
content_word_counts = pd.read_csv('../data/content_word_counts.csv', index_col='word')
solution_word_counts = pd.read_csv('../data/solution_word_counts.csv', index_col='word')
unique_words = set(list(content_word_counts.index) + list(solution_word_counts.index))

In [12]:
def significant_words(matrix):
    tf = matrix / matrix.sum()
    idf = np.log(2 / (matrix > 0).transpose().sum())
    tfidf = tf['incident'] * idf + tf['request'] * idf # @NOTE: Может есть лучше способ?
    return tfidf

all_significant_content_words = significant_words(content_word_counts[['incident', 'request']]).sort_values(ascending=False)
significant_content_words = list(all_significant_content_words.head(40).index)
significant_content_words += list(extract_words('консультация, поток, штат, успешно, сбой, инцидент, Сервис55, проблема, отчет, расписание, сброс, пароль, выполнено, работы, запрос, портал, доступ, разблокировка, заявка, обращение, сертификат, запись, код, установлено, перенесено, проинформирован, ошибка, пользователь, ne_boss, настроен, отменен, права, клиент, статус, справочник, тест, решен, файл, причина, договор'))
significant_content_words = np.unique(significant_content_words)
significant_content_words = list(set(significant_content_words) - (set(significant_content_words) - set(all_significant_content_words.index)))

all_significant_solution_words = significant_words(solution_word_counts[['incident', 'request']]).sort_values(ascending=False)
significant_solution_words = list(all_significant_solution_words.head(40).index)
significant_solution_words += list(extract_words('транспортный, системное, заблокировать, заявка, регистрация, учет, консультация, запрос, много, управление, бот, установка, изменение, настройка, письмо, доступ, код, учет, портал, выполнение, замена, устранение, ошибка, отзыв, контроль, сертификат, пользователь, ухудшение, создание, системное, клиент, прав, порт, сервер, запись, мест, ПО, ЭП, ПУ, блокировка'))
significant_solution_words = np.unique(significant_solution_words)
significant_solution_words = list(set(significant_solution_words) - (set(significant_solution_words) - set(all_significant_solution_words.index)))

print('Содержание:\n' + ', '.join(significant_content_words) + '\n\n' + 'Решение:\n' + ', '.join(significant_solution_words))

Содержание:
проблема, заявка, порт, отправлять, обновление, домен, статус, region, уза, запись, многочисленный, портал, сброс, организация, код, сбой, работа, новый, изменение, контроль, правый, доступ, замена, учет, ошибка, настраивать, пу, выполнять, блокировка, обращение, без, клиент, учётный, разблокировка, система91, файл, система364, консультация, устанавливать, логический, установка, инцидент, договор, право, ne, сертификат, причина, письмо, выполнение, пользователь, система54, пароль, boss, шлюз, система7, произойти, разблокировать, отзыв, справочник, эп, переносить, запрос, брм, отчет, удаление, тест, создание, учетный

Решение:
проинформировать, заблокировать, заявка, посредством, редактирование, нажатие, порт, добрый, сервер, тело, антивирус, ваш, идентификация, настройка, измениться, вопрос, ввод, день, запись, портал, необходимый, ухудшение, код, просьба, затем, изменение, контроль, системный, правый, доступ, проект, замена, учет, ошибка, дс, блокировка, клиент, место, ант

In [13]:
def prepare_x(data):
    X = pd.DataFrame(index=data.index)
    for word in significant_content_words:
        X['Содержание/' + word] = data['Содержание'].str.contains(word)
    X['Сервис'] = data['Сервис'].map(lambda x: service_dictionary.index(x))
    X['Приоритет'] = data['Приоритет'].str.extract(r'(\d+)')
    X['Статус'] = data['Статус'].map(lambda x: status_dictionary.index(x))
    X['Функциональная группа'] = data['Функциональная группа'].map(lambda x: fg_dictionary.index(x))
    X['Ожидаемое время исполнения, ч'] = (data['Крайний срок'] - data['Дата обращения']).dt.seconds / 360
    X['Фактическое время исполнения, ч'] = (data['Дата закрытия обращения'] - data['Дата обращения']).dt.seconds / 360
    X['Тип обращения на момент подачи'] = data['Тип обращения на момент подачи'].map(lambda x: type_dictionary.index(x))
    # Тип обращения итоговый
    for word in significant_solution_words:
        X['Решение/' + word] = data['Решение'].str.contains(word)
    # Тип переклассификации
    X['Критичность'] = data['Критичность'].str.extract(r'(\d+)')
    X['Влияние'] = data['Влияние'].str.extract(r'(\d+)')
    X['Система'] = data['Система'].map(lambda x: system_dictionary.index(x))
    X['Место'] = data['Место'].map(lambda x: location_dictionary.index(x))
    X = X.astype('float64')
    return X

x_data = prepare_x(train_data_clean)
x_test = prepare_x(test_data_clean)
x_stats = x_data.describe()

x_data = x_data / x_stats.loc['std']
x_test = x_test / x_stats.loc['std']

C:\Users\simple\AppData\Local\Temp\ipykernel_31864\560071462.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X['Решение/' + word] = data['Решение'].str.contains(word)
C:\Users\simple\AppData\Local\Temp\ipykernel_31864\560071462.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X['Решение/' + word] = data['Решение'].str.contains(word)
C:\Users\simple\AppData\Local\Temp\ipykernel_31864\560071462.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whic

In [14]:
x_data

,Содержание/проблема,Содержание/заявка,Содержание/порт,Содержание/отправлять,Содержание/обновление,Содержание/домен,Содержание/статус,Содержание/region,Содержание/уза,Содержание/запись,...,Решение/запрос,Решение/пункт,Решение/управление,Решение/карандаш,Решение/создание,Решение/транспортный,Критичность,Влияние,Система,Место
2,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,7.447015,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.750896,6.055512,2.625560,0.017493
3,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,3.003584,6.055512,0.000000,0.034985
4,0.0,2.022839,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,3.003584,8.074016,0.000000,0.052478
5,0.0,2.022839,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,3.003584,8.074016,2.560571,0.052478
6,0.0,2.022839,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,3.003584,8.074016,0.000000,0.052478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14798,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,3.631483,...,0.0,0.0,0.0,0.0,0.0,0.0,0.750896,6.055512,2.625560,1.311945
14799,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,3.003584,6.055512,0.000000,1.276960
14800,0.0,2.022839,4.277165,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,3.003584,8.074016,1.013830,0.612241
14801,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,3.003584,8.074016,2.651556,1.276960


In [15]:
y_data = pd.DataFrame(index=train_data_clean.index)
y_data['Тип переклассификации/0'] = train_data_clean['Тип переклассификации'] == 0
y_data['Тип переклассификации/1'] = train_data_clean['Тип переклассификации'] == 1
y_data['Тип переклассификации/2'] = train_data_clean['Тип переклассификации'] == 2
y_data = y_data.astype('float64')
y_data

,Тип переклассификации/0,Тип переклассификации/1,Тип переклассификации/2
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0
5,1.0,0.0,0.0
6,1.0,0.0,0.0
...,...,...,...
14798,1.0,0.0,0.0
14799,1.0,0.0,0.0
14800,1.0,0.0,0.0
14801,1.0,0.0,0.0


In [16]:
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size = 0.2, random_state=1337)
print(f"X shape: {x_train.shape}")
print(f"Y shape: {y_train.shape}")
print(f"Training = {len(x_train)}, validation = {len(x_val)}")

X shape: (11498, 153)
Y shape: (11498, 3)
Training = 11498, validation = 2875


In [21]:
model = keras.Sequential([
    keras.Input(shape=(x_train.shape[1])),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(40, activation="relu"),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(25, activation="relu"),
    # keras.layers.Dense(20, activation="relu"),
    # keras.layers.Dense(15, activation="relu"),
    # keras.layers.Dense(10, activation="relu"),
    # keras.layers.Dense(5, activation="relu"),
    keras.layers.Dense(3, activation="softmax"),
])

K = keras.backend

# @REFERENCE: https://www.kaggle.com/code/rejpalcz/best-loss-function-for-f1-score-metric
def f1_loss(y_true, y_pred):
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

model.compile(loss=f1_loss, optimizer='adamax', metrics=['accuracy'])

[initial_loss, initial_accuracy] = model.evaluate(x_train, y_train)
[initial_val_loss, initial_val_accuracy] = model.evaluate(x_val, y_val)
history = pd.DataFrame(dict(
    loss=[initial_loss],
    accuracy=[initial_accuracy],
    val_loss=[initial_val_loss],
    val_accuracy=[initial_val_accuracy],
    time=[0],
    batch_size=[math.nan],
))

model.summary()

90/90 [==============================] - 0s 1ms/step - loss: 0.8963 - accuracy: 0.0094
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 50)                7700      
                                                                 
 dense_5 (Dense)             (None, 40)                2040      
                                                                 
 dense_6 (Dense)             (None, 30)                1230      
                                                                 
 dense_7 (Dense)             (None, 25)                775       
                                                                 
 dense_8 (Dense)             (None, 3)                 78        
                                                                 
Total params: 11,823
Trainable params: 11,823
Non-trainable params: 0
_____________________________

In [38]:
%load_ext autoreload
%autoreload 2

from utils.time_callback import TimeCallback
from utils.metrics_callback import MetricsCallback

batch_size = 64
epochs = 200

time_callback = TimeCallback()
hist = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val), callbacks=[time_callback])
history_chunk = pd.merge(
    pd.DataFrame(hist.history),
    pd.DataFrame(dict(
        time=time_callback.times,
        batch_size=batch_size,
    )),
    left_index=True, right_index=True,
)
history = pd.concat((history, history_chunk), ignore_index=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Epoch 1/200
180/180 [==============================] - 0s 2ms/step - loss: 0.3624 - accuracy: 0.9915 - val_loss: 0.3812 - val_accuracy: 0.9850
Epoch 2/200
180/180 [==============================] - 0s 2ms/step - loss: 0.3818 - accuracy: 0.9914 - val_loss: 0.3810 - val_accuracy: 0.9854
Epoch 3/200
180/180 [==============================] - 0s 2ms/step - loss: 0.3671 - accuracy: 0.9917 - val_loss: 0.3798 - val_accuracy: 0.9854
Epoch 4/200
180/180 [==============================] - 0s 2ms/step - loss: 0.3755 - accuracy: 0.9917 - val_loss: 0.3809 - val_accuracy: 0.9854
Epoch 5/200
180/180 [==============================] - 0s 2ms/step - loss: 0.3802 - accuracy: 0.9913 - val_loss: 0.3853 - val_accuracy: 0.9857
Epoch 6/200
180/180 [==============================] - 0s 2ms/step - loss: 0.3905 - accuracy: 0.9912 - val_loss: 0.3829 - val_accuracy: 0.9843
Epoch 7/200
180/180 [==============================] -

In [43]:
px.line(history, y=['loss', 'val_loss'], log_y=True, log_x=True).show()

In [71]:
t_val = pd.DataFrame(index=y_val.index)
t_val[y_val.columns] = model.predict(x_val).astype('float64')
f1_score(np.argmax(t_val, axis=1), np.argmax(y_val, axis=1), average='macro')

90/90 [==============================] - 0s 907us/step


0.805472074631027

In [27]:
# SAVE_MODEL_PATH = '../models/tmp'
#
# model.save(f'{SAVE_MODEL_PATH}/model')
# x_stats.to_csv(f'{SAVE_MODEL_PATH}/stats.csv', index=True)
# with open(f'{SAVE_MODEL_PATH}/significant_content_words.txt', "w", encoding='utf-8') as outfile:
#     outfile.write("\n".join(str(item) for item in significant_content_words))
# with open(f'{SAVE_MODEL_PATH}/significant_solution_words.txt', "w", encoding='utf-8') as outfile:
#     outfile.write("\n".join(str(item) for item in significant_solution_words))
# x_stats.to_csv(f'{SAVE_MODEL_PATH}/stats.csv')
# history.to_csv(f'{SAVE_MODEL_PATH}/history.csv', index=False)

INFO:tensorflow:Assets written to: ../models/v1.big/model\assets


In [41]:
# LOAD_MODEL_PATH = '../models/v1.big'
# model = keras.models.load_model(f'{LOAD_MODEL_PATH}/model', custom_objects=dict(f1_loss=f1_loss))
# with open(f'{LOAD_MODEL_PATH}/significant_content_words.txt', "r", encoding='utf-8') as infile:
#     significant_content_words = list(map(lambda x: x.replace('\n', ''), infile.readlines()))
# with open(f'{LOAD_MODEL_PATH}/significant_solution_words.txt', "r", encoding='utf-8') as infile:
#     significant_solution_words = list(map(lambda x: x.replace('\n', ''), infile.readlines()))
# x_stats = pd.read_csv(f'{LOAD_MODEL_PATH}/stats.csv')
# history = pd.read_csv(f'{LOAD_MODEL_PATH}/history.csv')

In [72]:
t_test = pd.DataFrame(index=x_test.index)
t_test[y_val.columns] = model.predict(x_test).astype('float64')

def classify(initial, reclassify):
    if reclassify == 0:
        return initial
    if reclassify == 1:
        return 'Инцидент'
    if reclassify == 2:
        return 'Запрос'

submission = pd.DataFrame(index=x_test.index)
submission['Тип обращения на момент подачи'] = test_data_clean['Тип обращения на момент подачи']
submission['Тип переклассификации'] = np.argmax(t_test, axis=1)
submission['Тип обращения итоговый'] = [classify(initial, reclassify) for (initial, reclassify) in zip(test_data_clean['Тип обращения на момент подачи'], submission['Тип переклассификации'])]
submission['Тип переклассификации'][submission['Тип обращения на момент подачи'] == submission['Тип обращения итоговый']] = 0
del submission['Тип обращения на момент подачи']
submission

199/199 [==============================] - 0s 845us/step


C:\Users\simple\AppData\Local\Temp\ipykernel_31864\920904019.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Тип переклассификации,Тип обращения итоговый
0,0,Запрос
1,0,Запрос
2,0,Запрос
3,0,Запрос
4,0,Запрос
...,...,...
6339,0,Инцидент
6340,0,Инцидент
6341,0,Инцидент
6342,0,Инцидент
